In [6]:
import pickle
pathFolder = "./train/"
xTrainName = "winexTrain1.pkl"
yTrainName = "wineyTrain1.pkl"
with open(pathFolder+xTrainName,'rb') as f1:
    X = pickle.load(f1)
with open(pathFolder+yTrainName,'rb') as f2:
    Y = pickle.load(f2)

In [7]:
X_np = X.to_numpy()
Y_np = Y.to_numpy()

In [8]:
import numpy as np
import pandas as pd
# tile 함수를 사용하여 3차원 데이터 생성
# 예시로, 각 샘플을 3회 반복하여 차원을 확장
X_tiled_3d = np.tile(X_np, (1, 3)).reshape(X_np.shape[0], X_np.shape[1], 3)

# 결과 확인 (첫 샘플의 변환된 3차원 데이터)
X_tiled_3d[0]

array([[0.24778761, 0.39726027, 0.        ],
       [0.06849315, 0.10684474, 0.14084507],
       [0.09893993, 0.56754772, 0.60629921],
       [0.13772455, 0.15384615, 0.24778761],
       [0.39726027, 0.        , 0.06849315],
       [0.10684474, 0.14084507, 0.09893993],
       [0.56754772, 0.60629921, 0.13772455],
       [0.15384615, 0.24778761, 0.39726027],
       [0.        , 0.06849315, 0.10684474],
       [0.14084507, 0.09893993, 0.56754772],
       [0.60629921, 0.13772455, 0.15384615]])

In [9]:
import pickle

# xTrain.pkl 파일 로드
with open(r'C:\Users\bluecom011\Desktop\module_2\wine\xTrain.pkl', 'rb') as file:
    x_train = pickle.load(file)

# yTrain.pkl 파일 로드
with open(r'C:\Users\bluecom011\Desktop\module_2\wine\yTrain.pkl', 'rb') as file:
    y_train = pickle.load(file)

# 데이터 형태 확인
x_train.shape, y_train.shape


((1599, 11), (1599,))

In [3]:
#X_tiled_3d.shape

NameError: name 'X_tiled_3d' is not defined

In [10]:
# 간소화된 접근법: 전체 샘플을 224x224 형태로 확장하지 않고, 각 샘플의 특성 값을 이용하여 하나의 이미지로 변환
# 이번에는 각 샘플을 224x224 크기의 이미지로 만들고, 이를 RGB 채널로 복제하는 접근 방식을 시도
import numpy as np
# 특성 값을 14x14 타일로 확장하여 전체를 224x224 크기로 만듬 (14x14 타일로 11개 특성 사용, 나머지는 0으로 채움)
tiles_per_feature_expanded = 14  # 각 특성을 14x14 타일로 확장
num_features = x_train.shape[1]

# 샘플 데이터를 224x224 이미지로 변환 (나머지는 0으로 채움)
def transform_sample_to_image(sample):
    image = np.zeros((224, 224))  # 기본 이미지 틀
    for i in range(num_features):
        # 각 특성을 14x14 타일로 확장
        row = (i // 8) * tiles_per_feature_expanded
        col = (i % 8) * tiles_per_feature_expanded
        image[row:row+tiles_per_feature_expanded, col:col+tiles_per_feature_expanded] = np.tile(sample[i], (tiles_per_feature_expanded, tiles_per_feature_expanded))
    return image

# 첫 번째 샘플에 대해 변환 실행
first_sample_image = transform_sample_to_image(x_train[0])

# 첫 번째 샘플 이미지를 RGB로 변환
first_sample_image_rgb = np.repeat(first_sample_image[:, :, np.newaxis], 3, axis=2)

first_sample_image_rgb.shape


(224, 224, 3)

In [11]:
# 모든 샘플에 대해 이미지 변환 수행
transformed_images = np.array([transform_sample_to_image(sample) for sample in x_train])

# 각 이미지를 RGB로 변환
transformed_images_rgb = np.repeat(transformed_images[:, :, :, np.newaxis], 3, axis=3)

transformed_images_rgb.shape


(1599, 224, 224, 3)

In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# ResNet50 모델 로드 (사전 훈련된 가중치 사용, 최상위 층 제외)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 커스터마이징을 위해 학습 가능한 상태로 설정
for layer in base_model.layers:
    layer.trainable = True

# Global Average Pooling 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 사용자 정의 출력층 추가 (이진 분류)
predictions = Dense(1, activation='sigmoid')(x)

# 최종 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# 콜백 설정: 최적의 모델 저장 및 조기 중단
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# 데이터 분할: 학습 및 검증 데이터
X_train, X_val, y_train, y_val = train_test_split(transformed_images_rgb, y_train, test_size=0.2, random_state=42)

# 모델 학습
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=16, callbacks=[checkpoint, early_stopping])

history.history.keys()


Epoch 1/20
80/80 [==============================] - ETA: 0s - loss: 0.5997 - accuracy: 0.6998

c:\Users\bluecom011\miniconda3\envs\tensor2\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


80/80 [==============================] - 298s 4s/step - loss: 0.5997 - accuracy: 0.6998 - val_loss: 0.6884 - val_accuracy: 0.5594
Epoch 2/20
80/80 [==============================] - 327s 4s/step - loss: 0.5278 - accuracy: 0.7389 - val_loss: 0.6961 - val_accuracy: 0.4406
Epoch 3/20
80/80 [==============================] - 332s 4s/step - loss: 0.4993 - accuracy: 0.7584 - val_loss: 0.6978 - val_accuracy: 0.4406
Epoch 4/20
80/80 [==============================] - 318s 4s/step - loss: 0.5014 - accuracy: 0.7717 - val_loss: 0.6958 - val_accuracy: 0.4406
Epoch 5/20
80/80 [==============================] - 346s 4s/step - loss: 0.4779 - accuracy: 0.7952 - val_loss: 0.7020 - val_accuracy: 0.4406
Epoch 6/20
58/80 [====================>.........] - ETA: 1:26 - loss: 0.4422 - accuracy: 0.8039

KeyboardInterrupt: 

In [19]:
from sklearn.model_selection import train_test_split

# 예시 데이터: X, y
# X의 샘플 수: 1599, y의 샘플 수: 1599라고 가정

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 분할 후 샘플 수 확인
print(f"X_train 샘플 수: {len(X_train)}, y_train 샘플 수: {len(y_train)}")
print(f"X_test 샘플 수: {len(X_test)}, y_test 샘플 수: {len(y_test)}")


X_train 샘플 수: 1279, y_train 샘플 수: 1279
X_test 샘플 수: 320, y_test 샘플 수: 320


In [20]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
# 데이터 증강을 위한 레이어 설정
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# 기본 모델(ResNet50) 설정
IMG_SIZE = (160, 160)  # 이미지 크기 설정
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')

# 기본 모델의 가중치 고정
base_model.trainable = False

# 데이터 전처리 및 모델 레이어 설정
preprocess_input = tf.keras.applications.resnet50.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(3, activation='softmax')  # '3'은 예시, 실제 class_names의 길이에 따라 변경

# 모델 정의
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)  # 데이터 증강 적용
x = preprocess_input(x)        # 데이터 전처리
x = base_model(x, training=False)  # 기본 모델 적용
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)  # 드롭아웃 추가
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

# 모델 컴파일
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 학습률 스케줄러 설정
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)
# 데이터 분할: 학습 및 검증 데이터
x_train, x_val, y_train, y_val = train_test_split(transformed_images_rgb, y_train, test_size=0.2, random_state=42)
# 모델 학습
# train_dataset과 validation_dataset은 사전에 정의되어 있어야 합니다.
initial_epochs = 10
history = model.fit(x_train, y_train, 
                    epochs=initial_epochs,
                    validation_data=(x_val, y_val),
                    callbacks=[lr_scheduler_cb])

# 주의: train_dataset과 validation_dataset은 미리 준비된 TensorFlow 데이터셋이어야 하며,
# 해당 데이터셋은 이미지 및 레이블을 포함해야 합니다.


ValueError: Found input variables with inconsistent numbers of samples: [1599, 1279]

In [1]:
import matplotlib.pyplot as plt

# 학습 과정 시각화 (손실)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# 학습 과정 시각화 (정확도)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

NameError: name 'history' is not defined